In [9]:
import numpy as np
import math
import torch
import matplotlib.pyplot as plt
import gymnasium as gym
from gym import Env
from gym import spaces
import random
from IPython.display import clear_output
import os
import matplotlib
from collections import namedtuple, deque
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from pettingzoo import ParallelEnv
import seaborn as sns

is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

#device setup
device = "cpu"

import environment
from seq_buffer import Transition, SequentialReplayBuffer

In [10]:
freq_agile_radar_versions = [[9 * math.pow(10, 9), 9.4* math.pow(10, 9)], [8 * math.pow(10, 9), 8.5* math.pow(10, 9)], [6 * math.pow(10, 9), 6.2* math.pow(10, 9)]]

hop_states = []

for i in range (0,3):
    radar_states = []
    interval = (freq_agile_radar_versions[i][1] - freq_agile_radar_versions[i][0])/10
    for j in range(0,24):
        radar_states.append(int(freq_agile_radar_versions[i][0]+ interval * j))
    hop_states.append(radar_states)

jamming_bandwidths = [10e6, 25e6, 40e6]

In [11]:
env = environment.ParallelRadarJammingEnv(hop_states, jamming_bandwidths)

In [12]:
class VAE(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(VAE, self).__init__()
        self.input_dim = input_dim
        self.latent_dim = latent_dim

        # Encoder: maps the observation to mean and log-variance of the latent space
        self.fc1 = nn.Linear(input_dim, 128)
        self.fc2_mean = nn.Linear(128, latent_dim)
        self.fc2_log_var = nn.Linear(128, latent_dim)

        # Decoder: maps the latent vector back to the observation space
        self.fc3 = nn.Linear(latent_dim, 128)
        self.fc4 = nn.Linear(128, input_dim)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        mean = self.fc2_mean(h1)
        log_var = self.fc2_log_var(h1)
        return mean, log_var

    def reparameterize(self, mean, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mean + eps * std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))  # Using sigmoid to output values between 0 and 1

    def forward(self, x):
        mean, log_var = self.encode(x)
        z = self.reparameterize(mean, log_var)
        reconstructed_x = self.decode(z)
        return reconstructed_x, mean, log_var

    def loss_function(self, reconstructed_x, x, mean, log_var):
        # Reconstruction loss (binary cross-entropy)
        BCE = F.binary_cross_entropy(reconstructed_x, x, reduction='sum')

        # KL divergence between the learned distribution and the prior (normal distribution)
        # KL = 0.5 * sum(1 + log_var - mean^2 - exp(log_var))
        KL = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())
        return BCE + KL


In [13]:
class DIAL(nn.Module):
    def __init__(self, n_agents, latent_dim, message_dim, agent_ids):
        super(DIAL, self).__init__()
        
        self.n_agents = n_agents
        self.latent_dim = latent_dim
        self.message_dim = message_dim
        self.agent_ids = agent_ids
        
        # Define a message network for each agent
        self.message_network = nn.ModuleList([self.create_message_network() for _ in range(n_agents)])

    def create_message_network(self):
        # Ensure that the network input size is the sum of the latent and message dimensions
        return nn.Sequential(
            nn.Linear(self.latent_dim + self.message_dim, 64),  # Adjust based on input size
            nn.ReLU(),
            nn.Linear(64, self.message_dim)  # Output size is the message dimension
        )

    def aggregate_messages(self, all_latents, agent_id):
        # Collect latents from all agents except the current one
        other_agents = [latent for other_agent_id, latent in all_latents.items() if other_agent_id != agent_id]
        
        # Compute the aggregated message by averaging other agent latents
        aggregated_message = torch.stack(other_agents).mean(dim=0)
        
        # Concatenate the current agent's latent representation with the aggregated message
        message_input = torch.cat([all_latents[agent_id], aggregated_message], dim=-1)
        
        # Ensure the correct shape before passing it through the message network
        message = self.message_network[self.agent_ids.index(agent_id)](message_input)
        # Inside the aggregate_messages method:
        return message


In [14]:
class JammerAgent:
    def __init__(self, agent_id, vae, dial, latent_dim):
        self.agent_id = agent_id  # Unique ID for this agent
        self.vae = vae  # The VAE model for this agent
        self.dial = dial  # The DIAL module
        self.latent_dim = latent_dim  # Latent dimension size
    
    def get_latent_representation(self, observation):
        # Ensure the observation is a torch tensor (if it's a NumPy array or other type)
        observation_tensor = torch.tensor(observation, dtype=torch.float32).unsqueeze(0)  # Add batch dimension
        _, mean, log_var = self.vae(observation_tensor)
        return mean

    def receive_messages(self, all_latents, agent_id):
        # Use DIAL to aggregate messages from other agents
        return self.dial.aggregate_messages(all_latents, agent_id)

    def communicate_and_decide(self, observation, all_latents):
        # Get the latent representation from the VAE
        latent_rep = self.get_latent_representation(observation)

        # Receive and aggregate messages from other agents (using the dictionary format)
        received_message = self.receive_messages(all_latents, agent_id=self.agent_id)

        # Combine latent rep and received messages to form a new representation
        updated_latent_rep = latent_rep + received_message  # A simple combination, can be adjusted
        
        # Return the updated latent representation
        return updated_latent_rep


In [15]:
input_dim = sum(env.n_frequencies)
latent_dim = 24
message_dim = 24
learning_rate = 1e-3
n_agents = 6
batch_size = 2

agent_ids = [f'jammer_{i}' for i in range(len(env.agents))]

vae_models = {agent: VAE(input_dim=input_dim, latent_dim=latent_dim) for agent in env.agents}

dial_modules = {agent: DIAL(n_agents=len(env.agents), latent_dim=latent_dim, message_dim=message_dim, agent_ids=agent_ids)
                for agent in env.agents}

agents = {agent: JammerAgent(agent_id=agent, latent_dim=latent_dim, vae=vae_models[agent], dial=dial_modules[agent])
          for agent in env.agents}

optimizers = {agent: optim.Adam(agents[agent].vae.parameters(), lr=learning_rate) for agent in env.agents}

n_episodes = 1000
n_steps = 5

In [16]:
# Training loop
for episode in range(n_episodes):
    observations = env.reset()
    episode_rewards = {agent: 0 for agent in env.agents}

    # Initialize the replay buffer
    replay_buffer = SequentialReplayBuffer(maxlen=16000)

    for step in range(n_steps):
        latent_representations = {}
        for agent in env.agents:
            observation = torch.tensor(observations[agent], dtype=torch.float32)
            latent_representation = agents[agent].get_latent_representation(observation)
            latent_representations[agent] = latent_representation  # Store latent representations as a dictionary

        # Step 2: Communicate between agents using DIAL
        updated_latents = {}
        for agent in env.agents:
            updated_latents[agent] = agents[agent].communicate_and_decide(observations[agent], latent_representations)

        # Step 3: Take actions (example: random actions, replace with your action selection logic)
        actions = {agent: np.random.choice(env.action_space[agent].n) for agent in env.agents}

        # Step 4: Take the step in the environment
        next_observations, rewards, dones, infos = env.step(actions)

        # Step 5: Store the transitions in the replay buffer
        transitions = []
        for agent in env.agents:
            transition = Transition(
                state=observations[agent],
                action=actions[agent],
                next_state=next_observations[agent],
                reward=rewards[agent]
            )
            transitions.append(transition)

        replay_buffer.append(transitions)

        # Step 6: Sample from the replay buffer for training
        if len(replay_buffer.buffer) >= batch_size:
            sampled_sequences = replay_buffer.sample(batch_size=batch_size, sequence_length=100)
            
            # Process each sampled trajectory (sequence of transitions)
            for trajectory in sampled_sequences:
                for transition in trajectory:
                    for agent in env.agents:
                        observation = torch.tensor(transition.state, dtype=torch.float32).unsqueeze(0)
                        latent_representation = agents[agent].get_latent_representation(observation)

                        # Compute the loss and optimize the agent's VAE
                        reconstructed_x, mean, log_var = agents[agent].vae(observation)
                        loss = agents[agent].vae.loss_function(reconstructed_x, observation, mean, log_var)
                        optimizers[agent].zero_grad()
                        loss.backward()
                        optimizers[agent].step()

        # Track rewards for the episode
        for agent in env.agents:
            episode_rewards[agent] += rewards[agent]

        # Step 7: Print episode progress
        if step % 10 == 0:
            print(f"Episode {episode+1}/{n_episodes}, Step {step}/{n_steps}, Rewards: {episode_rewards}")
        
        # If all agents are done, break out of the loop
        if all(dones.values()):
            break

    # After each episode, we can log performance or make updates
    print(f"Episode {episode+1} finished with total rewards: {sum(episode_rewards.values())}")


C:\Users\Abhim\AppData\Local\Temp\ipykernel_32232\2487002653.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  observation_tensor = torch.tensor(observation, dtype=torch.float32).unsqueeze(0)  # Add batch dimension


Episode 1/1000, Step 0/5, Rewards: {'jammer_0': -55, 'jammer_1': -55, 'jammer_2': -55, 'jammer_3': 55, 'jammer_4': 85, 'jammer_5': -55}
torch.Size([1, 24]) torch.Size([1, 24])
torch.Size([1, 24]) torch.Size([1, 24])
torch.Size([1, 24]) torch.Size([1, 24])
torch.Size([1, 24]) torch.Size([1, 24])
torch.Size([1, 24]) torch.Size([1, 24])
torch.Size([1, 24]) torch.Size([1, 24])
torch.Size([1, 24]) torch.Size([1, 24])
torch.Size([1, 24]) torch.Size([1, 24])
torch.Size([1, 24]) torch.Size([1, 24])
torch.Size([1, 24]) torch.Size([1, 24])
torch.Size([1, 24]) torch.Size([1, 24])
torch.Size([1, 24]) torch.Size([1, 24])
torch.Size([1, 24]) torch.Size([1, 24])
torch.Size([1, 24]) torch.Size([1, 24])
torch.Size([1, 24]) torch.Size([1, 24])
torch.Size([1, 24]) torch.Size([1, 24])
torch.Size([1, 24]) torch.Size([1, 24])
torch.Size([1, 24]) torch.Size([1, 24])
torch.Size([1, 24]) torch.Size([1, 24])
torch.Size([1, 24]) torch.Size([1, 24])
torch.Size([1, 24]) torch.Size([1, 24])
torch.Size([1, 24]) torc

KeyboardInterrupt: 